In [72]:
import glob
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import coremltools
import cv2
%matplotlib notebook


def guided(img_y, mask):
    
    I = img_y.astype(np.float32) / 255.
    p = mask.astype(np.float32) / 255.
    
    min_size = min(p.shape)
    radius = int(min_size * 0.01)
    eps = 0.01
    
    q = np.clip(cv2.ximgproc.guidedFilter(I, p, radius, eps), 0, 1.)
    q2 = np.uint8(q * 255)
    
    return q2


def watershed(img_rgb, mask):
    
#     print(img_rgb.dtype)
#     print(mask.dtype)
    
    # Element
    kernel = np.ones((3,3),np.uint8)
    
    #
    min_size = min(mask.shape)
    it = int(min_size * 0.02)
#     print('iteration : %d' % it)

    # Sure foreground, background and unknown areas
    sure_fg = cv2.erode(mask, kernel, iterations=it)
    sure_bg = cv2.dilate(mask, kernel, iterations=it)
    unknown = sure_bg - sure_fg
    
    # Connected components
    ret, markers = cv2.connectedComponents(sure_fg)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1

    # Now, mark the region of unknown with zero
    markers[unknown>0] = 0
    
    # Apply the watershed
    markers2 = cv2.watershed(img_rgb, markers)

    # Keep only the foreground
    markers3 = np.clip(markers2 - 1, 0, 255)
    
    return markers3 * 255
    
#     f, ax = plt.subplots(1,2)
#     ax[0].imshow(mask)
#     ax[1].imshow(markers3)
#     plt.show()
    
    
    
    
def make_beautiful_image(img_rgb, mask):
    output = img_rgb
    red = output[:,:,0]
    red[mask > 127] = 255
    output[:,:,0] = red
    return output


    
    
    

# List of labels
labels = ['Background','Plane','Bicycle','Bird','Boat','Bottle','Bus','Car','Cat','Chair','Cow','Diningtable','Dog','Horse','Motorbike','Person','Pottedplant','Sheep','Sofa','Train','Tvmonitor']
label2id = {'Background':0,'Plane':1,'Bicycle':2,'Bird':3,'Boat':4,'Bottle':5,'Bus':6,'Car':7,'Cat':8,'Chair':9,'Cow':10,'Diningtable':11,'Dog':12,'Horse':13,'Motorbike':14,'Person':15,'Pottedplant':16,'Sheep':17,'Sofa':18,'Train':19,'Tvmonitor':20}


input_folder = '../../../samples'
mask_folder = 'output/mask/'
guided_folder = 'output/guided_filter/'
watershed_folder = 'output/watershed/'

# os.mkdir('output')
# os.mkdir(mask_folder)

if not os.path.isdir(mask_folder):
    os.makedirs(mask_folder)
if not os.path.isdir(guided_folder):
    os.makedirs(guided_folder)
if not os.path.isdir(watershed_folder):
    os.makedirs(watershed_folder)

# output_mask = 

paths = glob.glob(os.path.join(input_folder, '*.jpg'))
for path in paths:

    # File name
    name = os.path.split(path)[1]
    name_no_ext = os.path.splitext(name)[0]
    print('File name : ', name)

    # Read the image
    img_rgb = Image.open(path)
    img_y = img_rgb.convert('L')
#     plt.figure()
#     plt.imshow(img_rgb)
#     plt.imshow(img_y)
#     plt.show()

    # Load the model and it's specs
    model = coremltools.models.MLModel('../../../models/DeepLab.mlmodel')
    input_width = model.get_spec().description.input[0].type.imageType.width
    input_height = model.get_spec().description.input[0].type.imageType.height

    # Resize the image to fit model's input size
    small_img = img_rgb.resize((input_width, input_height), Image.BILINEAR)

    # Predict the classes
    y = model.predict({"image": small_img}, usesCPUOnly=False)
    small_pred = y['scores']

    # Resize the prediction to input image size
    pred = small_pred.resize((img_rgb.width, img_rgb.height), PIL.Image.NEAREST)

#     # Plot
#     f, ax = plt.subplots(1,3)
#     ax[0].imshow(img_rgb)
#     ax[1].imshow(img_y, cmap='gray')
#     ax[2].imshow(pred)
#     ax[0].set_axis_off()
#     ax[1].set_axis_off()
#     ax[2].set_axis_off()
#     plt.show()

    # Convert images
#     guide = np.array(img_y).astype(np.float32) / 255.
    pred2 = np.array(pred) / 12

    # Print statistics
    nb_pixels = pred2.size
    for label in range(len(labels)):
        
        nb_pixels_for_class = float(len(np.nonzero(pred2 == label)[0]))
        percentage_class = nb_pixels_for_class / nb_pixels
#         print("  %-12s : %4.1f %%" % (labels[i], 100 * percentage_class))

        # Treat only represented classes
        if percentage_class > 0.01:
        
        # Specify the class label you are interested in
        # label = 'Person'

            # Compute the associated mask
            mask = np.zeros(pred2.shape, dtype=np.uint8)
            mask[pred2 == label] = 255
            
            # Red filter
            beautiful = make_beautiful_image(np.array(img_rgb), mask)
            
#             plt.figure()
# #             plt.imshow(mask)
#             plt.imshow(beautiful)
#             plt.show()

            # Save the mask
            mask_name = os.path.join(os.path.join(mask_folder, '%s.%s.png' % (name_no_ext, labels[label])))
            Image.fromarray(beautiful).save(mask_name, "PNG")
#             print(mask_name)
            
#             print(np.array(img_y).dtype)
    
            # Guided
            q = guided(np.array(img_y), mask)
            beautiful_q = make_beautiful_image(np.array(img_rgb), q)
#             plt.figure()
# #             plt.imshow(mask)
# #             plt.imshow(q)
#             plt.imshow(beautiful_q)
#             plt.show()
            mask_name = os.path.join(os.path.join(guided_folder, '%s.%s.png' % (name_no_ext, labels[label])))
            Image.fromarray(beautiful_q).save(mask_name, "PNG")
        
            wat = watershed(np.array(img_rgb), mask)
            beautiful_wat = make_beautiful_image(np.array(img_rgb), wat)
#             plt.figure()
# #             plt.imshow(mask)
# #             plt.imshow(q)
# #             plt.imshow(wat)
#             plt.imshow(beautiful_wat)
#             plt.show()
            mask_name = os.path.join(os.path.join(watershed_folder, '%s.%s.png' % (name_no_ext, labels[label])))
            Image.fromarray(beautiful_wat).save(mask_name, "PNG")

    
    

#     break

('File name : ', 'dog.jpg')
('File name : ', 'horse.jpg')
('File name : ', 'boat.jpg')
('File name : ', 'cat_woman.jpg')
('File name : ', 'woman_car_3.jpg')
('File name : ', 'flowers.jpg')
('File name : ', 'plane.jpg')
('File name : ', 'person_woman.jpg')
('File name : ', 'frame0008.jpg')
('File name : ', 'motorbike.jpg')
('File name : ', 'bike.jpg')
('File name : ', 'sheep.jpg')
('File name : ', 'bird.jpg')
('File name : ', 'moto_man.jpg')
('File name : ', 'potted_plant.jpg')
('File name : ', 'cow.jpg')
('File name : ', 'person.jpg')
('File name : ', 'bottle.jpg')
('File name : ', 'dog_guinness.jpg')
('File name : ', 'sofa.jpg')
('File name : ', 'cat.jpg')
('File name : ', 'chair.jpg')
('File name : ', 'minivan_2.jpg')
('File name : ', 'dining_table.jpg')
('File name : ', 'car.jpg')
('File name : ', 'person_matthieu.jpg')
('File name : ', 'tv_monitor.jpg')
('File name : ', 'person_woman_2.jpg')
('File name : ', 'bus.jpg')
('File name : ', 'woman_car.jpg')
('File name : ', 'minivan.jpg